In [11]:
from keras.models import load_model
from keras.preprocessing import sequence

In [16]:
testing_data_path = '/home/csdai/hw4_data/testing_data.txt'
testing_data_index_path = '/home/csdai/hw4_data/testing_data_index.txt'
dict_path = '/home/csdai/hw4_data/dict.txt'
model_path = '/home/csdai/hw4_data/model/'
answer_path = '/home/csdai/hw4_data/answer.txt'
max_review_length = 30

In [17]:
def read_dict(dict_path):
    d = []
    with open(dict_path, 'r') as dict_txt:
        for line in dict_txt:
            d.append(line.strip())
    return d

def indexing_testing_data(testing_data_path, testing_data_index_path, dict_path):
    d = read_dict(dict_path)
    with open(testing_data_path, 'r') as test_data:
        all_index_text = []
        test_data.readline()
        counter = 0
        for line in test_data:
            counter += 1
            if (counter % 20000) == 0:
                print(counter)
            first_comma = line.index(',')
            index = line.strip()[:first_comma]
            text = line.strip()[first_comma+1:]
            index_text = ''
            words = text.split(' ')
            for word in words:
                if word in d:
                    index_text += str(d.index(word) + 1) +' '
                else:
                    index_text += str(1) +' ' 
            all_index_text.append(index_text.strip())
        if os.path.isfile(testing_data_index_path):
            os.remove(testing_data_index_path)
        with open(testing_data_index_path, 'a') as index_data:
            index = 0
            for index_text in all_index_text:
                index_data.write(str(index) +','+ index_text +'\r\n')
                index += 1
    return None

def read_test_data(testing_data_index_path):
    x = []
    with open(testing_data_index_path, 'r') as test_data:
        for line in test_data:
            index, text = line.strip().split(',')
            x.append(list(map(int, text.split(' '))))
    return x

def Q3_RNN(dict_path):
    d = read_dict(dict_path)
    model = load_model(model_path + 'model_7.h5')
    test_str = ["today is a good day , but it is hot", "today is hot , but it is a good day"]
    all_index_text = []
    for text in test_str:
        index_text = []
        words = text.split(' ')
        for word in words:
            if word in d:
                index_text.append(d.index(word) + 1)
            else:
                index_text.append(1) 
    
        all_index_text.append(index_text)
    pred = model.predict(sequence.pad_sequences(all_index_text, maxlen=max_review_length, padding='post'))
    return pred
    
def write_answer(pred, answer_path):
    if os.path.isfile(answer_path):
        os.remove(answer_path)
    with open(answer_path, 'a') as answer:
        index = 0
        answer.write('id,label\r\n')
        for score in pred:
            if score >= 0.5:
                answer.write(str(index) +',1\r\n')
            else:
                answer.write(str(index) +',0\r\n')
            index += 1 

def main():
    indexing_testing_data(testing_data_path, testing_data_index_path, dict_path)
    read_test_data(testing_data_index_path)
    pred = model.predict(x_test)
    write_answer(pred, answer_path)
    
print(Q3_RNN(dict_path))

[[ 0.41614577]
 [ 0.95836008]]
